In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import sys
import math
import time
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import lib.dataset as dataset
import lib.params as params
import lib.network as network
import lib.path as path
import lib.utils as utils
from datetime import datetime

In [3]:
# read params
with open("config/train.params") as f:
    learning_rate=float(params.read_param(f.readline()))
    batch_size=int(params.read_param(f.readline()))
    epoch=int(params.read_param(f.readline()))
   
    print("training with a learning rate of {} for {} epochs with batchs of size {}".format(learning_rate,epoch,batch_size))

    
data_all=np.array(sorted(path.construct_path_lists("out","data_")))
label_all=np.array(sorted(path.construct_path_lists("out","labels_")))
# print(data_all.shape,label_all.shape)

print("creating network...")
net=network.R2N2(learning_rate)
print(" ...network created")

training with a learning rate of 0.1 for 1 epochs with batchs of size 3
creating network...
encoder_network
recurrent_module
decoder_network
setup loss function
initializing network
 ...network created


In [4]:
i=1
x=utils.to_npy(data_all[i])
y=utils.to_npy(label_all[i])

In [5]:
s=net.get_states(x,y)

In [15]:
len(s)

39

In [27]:
np.s[-1]

array([[[[[  5.05194208e-03,   0.00000000e+00],
          [  0.00000000e+00,   0.00000000e+00],
          [  2.38137599e-03,   0.00000000e+00],
          ..., 
          [  0.00000000e+00,   0.00000000e+00],
          [  5.81322052e-03,   6.52975170e-04],
          [  0.00000000e+00,   2.57057277e-03]],

         [[  0.00000000e+00,   1.80193828e-03],
          [  1.09630227e-02,   0.00000000e+00],
          [  1.91203249e-03,   1.82487676e-03],
          ..., 
          [  0.00000000e+00,   1.66459568e-02],
          [  7.04177562e-03,   0.00000000e+00],
          [  5.09000337e-03,   6.26302045e-03]],

         [[  3.33061395e-03,   0.00000000e+00],
          [  0.00000000e+00,   1.19191781e-03],
          [  1.80390291e-03,   3.17872427e-02],
          ..., 
          [  0.00000000e+00,   0.00000000e+00],
          [  0.00000000e+00,   9.52388905e-03],
          [  1.03744157e-02,   0.00000000e+00]],

         ..., 
         [[  6.36553112e-03,   8.46560672e-03],
          [  0.0000

In [ ]:
model_dir="out/model_{}_{}_{} {}".format(learning_rate,epoch,batch_size,datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))
    
# train network
loss_all=[]
acc_all=[]
print("starting training")
for e in range(epoch):
    start_time=time.time()
    loss_epoch=[]
    batch_number=0
    data_batchs,label_batchs=utils.get_batchs(data_all,label_all,batch_size)
    for data,label in zip(data_batchs,label_batchs):
        loss_epoch.append(net.train_step(data,label))
        print("epoch_{:03d}-batch_{:03d}: loss={}".format(e,batch_number,loss_epoch[-1]))
        batch_number+=1
    loss_all.append(loss_epoch)
    net.save("{}/epoch_{:03d}".format(model_dir,e),"loss",loss_all)
    print("epoch %d took %d seconds"%(e,time.time()-start_time))